In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
# Good data
df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [40]:
# Messy data
df = pd.read_csv('./data/affcon_final_politeness_strategies_merge.csv')

In [41]:
df = df.fillna(0)

In [106]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport_c = to_categorical(train['affcon_rapport'].tolist())
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport_c = to_categorical(test['affcon_rapport'].tolist())
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

### Individual classifiers are Random Forests

In [83]:
# Game move classifier
clf_gamemove = RandomForestClassifier(max_depth=2, random_state=0)
clf_gamemove.fit(X_train, y_train_gamemove)
y_pred_gamemove = clf_gamemove.predict(X_train)
y_pred_test_gamemove = clf_gamemove.predict(X_test)
print(classification_report(y_pred_gamemove, y_train_gamemove))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.84      0.91      3210

    accuracy                           0.84      3210
   macro avg       0.50      0.42      0.46      3210
weighted avg       1.00      0.84      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [107]:
# Rapport classifier
clf_rapport = RandomForestClassifier(max_depth=2, random_state=0)
clf_rapport.fit(X_train, y_train_rapport)
y_pred_rapport = clf_rapport.predict(X_train)
y_pred_test_rapport = clf_rapport.predict(X_test)
print(classification_report(y_pred_rapport, y_train_rapport))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.83      0.91      3210

    accuracy                           0.83      3210
   macro avg       0.50      0.42      0.45      3210
weighted avg       1.00      0.83      0.91      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
# Share Information classifier
clf_shareinfo = RandomForestClassifier(max_depth=2, random_state=0)
clf_shareinfo.fit(X_train, y_train_share_information)
y_pred_shareinfo = clf_shareinfo.predict(X_train)
y_pred_test_shareinfo = clf_shareinfo.predict(X_test)
print(classification_report(y_pred_shareinfo, y_train_share_information))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.81      0.89      3210

    accuracy                           0.81      3210
   macro avg       0.50      0.40      0.45      3210
weighted avg       1.00      0.81      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
# Reasoning classifier
clf_reasoning = RandomForestClassifier(max_depth=2, random_state=0)
clf_reasoning.fit(X_train, y_train_reasoning)
y_pred_reasoning = clf_reasoning.predict(X_train)
y_pred_test_reasoning = clf_reasoning.predict(X_test)
print(classification_report(y_pred_reasoning, y_train_reasoning))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.80      0.89      3210

    accuracy                           0.80      3210
   macro avg       0.50      0.40      0.44      3210
weighted avg       1.00      0.80      0.89      3210



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
pred_df_arr = []
for i in range(0, len(y_pred_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_gamemove[i]
    pred_obj['reasoning'] = y_pred_reasoning[i]
    pred_obj['shareinfo'] = y_pred_shareinfo[i]
    pred_obj['rapport'] = y_pred_rapport[i]
    
    pred_df_arr.append(pred_obj)
    
pred_df = pd.DataFrame(pred_df_arr)

In [110]:
pred_test_df_arr = []
for i in range(0, len(y_pred_test_gamemove)):
    pred_obj = {}
    pred_obj['gamemove'] = y_pred_test_gamemove[i]
    pred_obj['reasoning'] = y_pred_test_reasoning[i]
    pred_obj['shareinfo'] = y_pred_test_shareinfo[i]
    pred_obj['rapport'] = y_pred_test_rapport[i]
    
    pred_test_df_arr.append(pred_obj)
    
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [51]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [112]:
# Combined model

inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
m = Model(inputs=inputB, outputs=c)

m.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m, precision_m,recall_m])
history = m.fit(x=pred_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_deception), callbacks=[callback])

Epoch 1/32
51/51 [==============================] - 0s 8ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/32
51/51 [==============================] - 0s 4ms/step - loss: 7.7125 - acc: 0.0458 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.0523 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00


### Individual Classifiers are MLP

In [113]:
y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport = to_categorical(train['affcon_rapport'].tolist())
y_train_share_information = to_categorical(train['affcon_shareinformation'].tolist())
y_train_reasoning = to_categorical(train['affcon_reasoning'].tolist())
y_train_gamemove = to_categorical(train['affcon_gamemove'].tolist())

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport = to_categorical(test['affcon_rapport'].tolist())
y_test_share_information = to_categorical(test['affcon_shareinformation'].tolist())
y_test_reasoning = to_categorical(test['affcon_reasoning'].tolist())
y_test_gamemove = to_categorical(test['affcon_gamemove'].tolist())

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

In [114]:
# Game move model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=X_train, y=y_train_gamemove, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test, y_test_gamemove, verbose=0)
print(precision, recall, f1_score)

y_pred_gamemove = gamemove_model.predict(X_train)
y_pred_test_gamemove = gamemove_model.predict(X_test)

Epoch 1/32
41/41 [==============================] - 0s 8ms/step - loss: 4.4695 - acc: 0.8357 - f1_m: 0.5196 - precision_m: 0.5271 - recall_m: 0.5343 - val_loss: 3.3540 - val_acc: 0.8489 - val_f1_m: 0.4062 - val_precision_m: 0.5493 - val_recall_m: 0.3281
Epoch 2/32
41/41 [==============================] - 0s 4ms/step - loss: 3.0580 - acc: 0.8431 - f1_m: 0.3630 - precision_m: 0.5515 - recall_m: 0.2763 - val_loss: 3.0895 - val_acc: 0.8489 - val_f1_m: 0.4206 - val_precision_m: 0.5631 - val_recall_m: 0.3409
Epoch 3/32
41/41 [==============================] - 0s 4ms/step - loss: 2.7283 - acc: 0.8431 - f1_m: 0.2841 - precision_m: 0.5674 - recall_m: 0.2069 - val_loss: 2.3662 - val_acc: 0.8474 - val_f1_m: 0.1205 - val_precision_m: 0.5567 - val_recall_m: 0.0682
Epoch 4/32
41/41 [==============================] - 0s 4ms/step - loss: 2.4846 - acc: 0.8438 - f1_m: 0.2330 - precision_m: 0.6070 - recall_m: 0.1521 - val_loss: 2.5463 - val_acc: 0.8489 - val_f1_m: 0.3652 - val_precision_m: 0.6157 - val_r

In [ ]:
# Reasoning model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=X_train, y=y_train_gamemove, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test, y_test_gamemove, verbose=0)
print(precision, recall, f1_score)

y_pred_gamemove = gamemove_model.predict(X_train)
y_pred_test_gamemove = gamemove_model.predict(X_test)

### Individual classifiers are LSTMs

Merge classifiers by Add

In [42]:
train, test = train_test_split(df, test_size=0.2)

y_train_deception = train['Input.deception_quadrant_cat'].tolist()
y_train_rapport = train['affcon_rapport'].tolist()
y_train_share_information = train['affcon_shareinformation'].tolist()
y_train_reasoning = train['affcon_reasoning'].tolist()
y_train_gamemove = train['affcon_gamemove'].tolist()

y_test_deception = test['Input.deception_quadrant_cat'].tolist()
y_test_rapport = test['affcon_rapport'].tolist()
y_test_share_information = test['affcon_shareinformation'].tolist()
y_test_reasoning = test['affcon_reasoning'].tolist()
y_test_gamemove = test['affcon_gamemove'].tolist()

X_train_col = train['Input.full_text']

X_test_col = test['Input.full_text']

In [5]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [43]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [44]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [45]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

In [46]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.6695 - accuracy: 0.6064 - f1_m: 0.7537 - recall_m: 0.9962 - precision_m: 0.6073 - val_loss: 0.6685 - val_accuracy: 0.5956 - val_f1_m: 0.7434 - val_recall_m: 1.0000 - val_precision_m: 0.5932
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.6485 - accuracy: 0.6091 - f1_m: 0.7456 - recall_m: 0.9509 - precision_m: 0.6155 - val_loss: 0.6772 - val_accuracy: 0.5953 - val_f1_m: 0.6982 - val_recall_m: 0.7992 - val_precision_m: 0.6229


In [55]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [56]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.6045 - accuracy: 0.7138 - f1_m: 0.8296 - recall_m: 0.9917 - precision_m: 0.7177 - val_loss: 0.5857 - val_accuracy: 0.7268 - val_f1_m: 0.8414 - val_recall_m: 1.0000 - val_precision_m: 0.7265
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5864 - accuracy: 0.7172 - f1_m: 0.8345 - recall_m: 0.9998 - precision_m: 0.7171 - val_loss: 0.5789 - val_accuracy: 0.7271 - val_f1_m: 0.8415 - val_recall_m: 1.0000 - val_precision_m: 0.7268
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5773 - accuracy: 0.7176 - f1_m: 0.8344 - recall_m: 0.9973 - precision_m: 0.7183 - val_loss: 0.5827 - val_accuracy: 0.7259 - val_f1_m: 0.8402 - val_recall_m: 0.9952 - val_precision_m: 0.7274


In [57]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [58]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 41ms/step - loss: 0.5519 - accuracy: 0.7645 - f1_m: 0.8648 - recall_m: 0.9946 - precision_m: 0.7675 - val_loss: 0.5474 - val_accuracy: 0.7519 - val_f1_m: 0.8579 - val_recall_m: 1.0000 - val_precision_m: 0.7521
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.5234 - accuracy: 0.7681 - f1_m: 0.8683 - recall_m: 0.9974 - precision_m: 0.7698 - val_loss: 0.5544 - val_accuracy: 0.7525 - val_f1_m: 0.8580 - val_recall_m: 0.9992 - val_precision_m: 0.7529


In [59]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [60]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.6826 - accuracy: 0.5500 - f1_m: 0.5716 - recall_m: 0.6715 - precision_m: 0.5619 - val_loss: 0.6924 - val_accuracy: 0.5569 - val_f1_m: 0.5957 - val_recall_m: 0.6263 - val_precision_m: 0.5706
Epoch 2/15
99/99 [==============================] - 3s 34ms/step - loss: 0.6413 - accuracy: 0.6234 - f1_m: 0.6067 - recall_m: 0.5718 - precision_m: 0.6666 - val_loss: 0.7182 - val_accuracy: 0.5362 - val_f1_m: 0.5330 - val_recall_m: 0.5086 - val_precision_m: 0.5633


In [61]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [62]:
# Deception model
deception_model = create_lstm()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
deception_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 4s 40ms/step - loss: 0.2267 - accuracy: 0.9421 - f1_m: 0.9663 - recall_m: 0.9907 - precision_m: 0.9512 - val_loss: 0.2195 - val_accuracy: 0.9444 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9440
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.2021 - accuracy: 0.9502 - f1_m: 0.9745 - recall_m: 1.0000 - precision_m: 0.9504 - val_loss: 0.2187 - val_accuracy: 0.9444 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9440
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.1966 - accuracy: 0.9502 - f1_m: 0.9745 - recall_m: 1.0000 - precision_m: 0.9504 - val_loss: 0.2197 - val_accuracy: 0.9444 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9440


In [31]:
# One hot encoding
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred[i][0]
    pred_obj['reasoning'] = reasoning_pred[i][0]
    pred_obj['shareinfo'] = shareinfo_pred[i][0]
    pred_obj['rapport'] = rapport_pred[i][0]
    
    pred_df_arr.append(pred_obj)
    
pred_df = pd.DataFrame(pred_df_arr)

In [32]:
pred_test_df_arr = []
for i in range(0, len(gamemove_pred_test)):
    pred_obj = {}
    pred_obj['gamemove'] = gamemove_pred_test[i][0]
    pred_obj['reasoning'] = reasoning_pred_test[i][0]
    pred_obj['shareinfo'] = shareinfo_pred_test[i][0]
    pred_obj['rapport'] = rapport_pred_test[i][0]
    
    pred_test_df_arr.append(pred_obj)
    
pred_test_df = pd.DataFrame(pred_test_df_arr)

In [34]:
# Joint Model with one hot encoding 
inputB = Input(shape=(pred_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=pred_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df, y_test_deception), callbacks=[callback])

Epoch 1/32
51/51 [==============================] - 0s 8ms/step - loss: 14.6849 - acc: 0.0480 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 14.7526 - val_acc: 0.0436 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/32
51/51 [==============================] - 0s 4ms/step - loss: 14.6849 - acc: 0.0480 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 14.7526 - val_acc: 0.0436 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00


In [63]:
# Joint Model with model Add
commonInput = Input(shape=[max_len])

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
out4 = rapport_model(commonInput)
mergedOut = Add()([out1,out2,out3,out4])

mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(256, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(128, activation='relu')(mergedOut)
mergedOut = Dropout(.35)(mergedOut)
mergedOut = Dense(2, activation='softmax')(mergedOut)  #Cuz binary

mergedModel = Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
99/99 [==============================] - 3s 35ms/step - loss: 0.2219 - accuracy: 0.9438 - f1_m: 0.9699 - recall_m: 0.9924 - precision_m: 0.9504 - val_loss: 0.2190 - val_accuracy: 0.9444 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9440
Epoch 2/15
99/99 [==============================] - 3s 35ms/step - loss: 0.2005 - accuracy: 0.9502 - f1_m: 0.9743 - recall_m: 1.0000 - precision_m: 0.9501 - val_loss: 0.2156 - val_accuracy: 0.9444 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9440
Epoch 3/15
99/99 [==============================] - 3s 35ms/step - loss: 0.1898 - accuracy: 0.9502 - f1_m: 0.9743 - recall_m: 0.9998 - precision_m: 0.9503 - val_loss: 0.2402 - val_accuracy: 0.9444 - val_f1_m: 0.9710 - val_recall_m: 1.0000 - val_precision_m: 0.9440
